In [11]:
%pip install numpy pandas nltk
import nltk
nltk.download('stopwords')


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/atharvchandratre/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

In [13]:
data = pd.read_csv('../dataset/dataset.csv', sep='|')
data.Reviews=data.Reviews.astype(str)

#Transform text to lowercase
data['Reviews'] = data['Reviews'].apply(lambda x: x.lower()) 
#Removing all punctuations and special characters
data['Reviews'] = data['Reviews'].apply(lambda x: re.sub('[,.]', '', x))

In [14]:
stopwords_list = stopwords.words('english')
data['Reviews'] = data['Reviews'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords_list)]))

In [15]:
print(data)

                   Company                                            Reviews
0                  Netflix  high performance culture challenge netflix ask...
1                     Uber  pay gone years first started pay great! gas pr...
2                     Yext  excellent ceo -lucky worked given opportunity ...
3                     CACI  nice place work great company work benefits gr...
4            Covenant Eyes  best company i’ve ever worked covenant eyes ca...
..                     ...                                                ...
117  Reynolds and Reynolds  5 stars best part working company? management ...
118                  Sabre  easy going workplace “you always deliver work ...
119  Legrand North America  best place work managers jobs correctly always...
120    Samsung Electronics  interesting challenging work one favorite comp...
121         Teladoc Health  well-organized inclusive excellent management ...

[122 rows x 2 columns]


In [16]:
from textblob import TextBlob

In [17]:
score_list=[]

for text in data['Reviews']:
  
  sentence = TextBlob(text)
  
  score = sentence.sentiment.polarity
  score_list.append(score)

In [18]:
import statistics
median = statistics.median(score_list)
print(median)

0.24523521411378624


In [19]:
sentiment_label = []
for score in score_list:
  if score > median:
    sentiment_label.append('positive')
  elif score < median:
    sentiment_label.append('negative')
  else:
    sentiment_label.append('neutral')
print(sentiment_label)

['positive', 'negative', 'negative', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'positive', 'negative', 'positive', 'positive', 'positive', 'negative', 'positive', 'negative', 'negative', 'positive', 'positive', 'positive', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'positive', 'positive', 'positive', 'positive', 'positive', 'negative', 'positive', 'positive', 'positive', 'negative', 'negative', 'positive', 'positive', 'positive', 'positive', 'negative', 'negative', 'positive', 'negative', 'positive', 'positive', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'positive', 'positive', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'positive', 'po

In [20]:
dataset = data
dataset['SCORE']=score_list
dataset['SENTIMENT_LABEL']=sentiment_label
print(dataset)
dataset.to_csv("../dataset/TextBlobResultswithMedian.csv", sep='|')

                   Company                                            Reviews  \
0                  Netflix  high performance culture challenge netflix ask...   
1                     Uber  pay gone years first started pay great! gas pr...   
2                     Yext  excellent ceo -lucky worked given opportunity ...   
3                     CACI  nice place work great company work benefits gr...   
4            Covenant Eyes  best company i’ve ever worked covenant eyes ca...   
..                     ...                                                ...   
117  Reynolds and Reynolds  5 stars best part working company? management ...   
118                  Sabre  easy going workplace “you always deliver work ...   
119  Legrand North America  best place work managers jobs correctly always...   
120    Samsung Electronics  interesting challenging work one favorite comp...   
121         Teladoc Health  well-organized inclusive excellent management ...   

        SCORE SENTIMENT_LAB